In [6]:
def bits2dec(bits: str, intdigit: int):
    int_bits = bits[:intdigit]
    float_bits = bits[intdigit:]
    
    num_int: int = 0
    num_float = 0
    
    for i in int_bits:
        num_int = num_int << 1
        num_int += int(i)
    
    for f in float_bits[::-1]:
        num_float += int(f)
        num_float /= 2
        
    return num_int + num_float

bits2dec("11011", 2)

3.375

In [55]:
import random

a = bin(random.randint(1, 100))
b = bin(random.randint(1, 100))

print(a, b)

shift = 20

a_int = int(a, 2)
b_int = int(b, 2)

print(a_int, b_int)
print(a_int / b_int)

c_int = int((a_int << shift) / b_int)
print(bin(c_int)[2:])
c_bits_str = "0" * (len(a) + shift - len(bin(c_int)[2:])) + bin(c_int)[2:]
print(c_bits_str, len(c_bits_str), len(a))
print(bits2dec(c_bits_str, len(a)))

0b1000101 0b10001
69 17
4.0588235294117645
10000001111000011110000
00000010000001111000011110000 29 9
4.0588226318359375


In [7]:
import random

a = random.random() * 100
b = random.random() * 100

In [8]:
a, b

(85.65160708098149, 74.93887171092261)

In [9]:
a * b

6418.634794876023

In [10]:
# 8 bits int, 8 bits floating points
a = int(a * 2 ** 8)
b = int(b * 2 ** 8)

In [11]:
hex(a), hex(b)

('0x55a6', '0x4af0')

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

from tqdm import tqdm
import logging
import pandas as pd

import os
import sys

mean = [
    773.01792745,
    -88.91943195,
    257.42027559,
    318.24409449,
    324.74732846,
    334.68848425,
    340.11283746,
    307.42055681,
]
std = [
    497.12221911,
    253.92590509,
    272.79684578,
    19.25624324,
    45.58375268,
    43.85550165,
    36.24868204,
    41.05464977,
]


class NMSLDataset(Dataset):
    def __init__(self, data_path: str):
        data = []
        labels = []

        speakers = os.listdir(data_path)

        for i in speakers:
            labeled_data_files = os.listdir(os.path.join(data_path, i))
            for file in labeled_data_files:
                with open(os.path.join(data_path, i, file), "r") as f:
                    for line in f.readlines()[:-1]:
                        line = line.strip().split(" ")
                        v = [int(i) for i in line[:]]

                        data.append(v)
                        if file[0] in [chr(ord("A") + i) for i in range(26)]:
                            labels.append(torch.tensor(ord(file[0]) - ord("A")))
                        elif "halt" in file:
                            labels.append(torch.tensor(26))
                        elif "random" in file:
                            labels.append(torch.tensor(27))

        data = np.array(data)
        # normalize data
        self.mean = np.mean(data, axis=0)
        self.std = np.std(data, axis=0)
        data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)

        self.data = []
        self.labels = []

        this_data = []
        this_label = -1
        for i, v in enumerate(data):
            if len(this_data) < 5:
                this_data.append(v)
                this_label = labels[i]
            else:
                self.data.append(torch.tensor(this_data, dtype=torch.float).T)
                self.labels.append(torch.tensor(this_label))

                if labels[i] != this_label:
                    this_data = []
                    this_label = -1
                    continue
                else:
                    this_data.pop(0)
                    this_data.append(v)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

    def __len__(self):
        return len(self.data)

# define model
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(8, 10, 3),
            nn.BatchNorm1d(10),
            nn.Flatten(),
        )

        self.fc = nn.Sequential(
            nn.Linear(10 * 3, 27),
        )

    def forward(self, x):
        output = self.cnn(x)
        output = self.fc(output)
        return output

In [15]:
train_set = NMSLDataset("../dataset/train")
train_loader = DataLoader(train_set, batch_size=1)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = Classifier().to(device)
model.load_state_dict(torch.load("./models/model.pth"))

/tmp/ipykernel_168618/455989549.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels.append(torch.tensor(this_label))


<All keys matched successfully>

In [18]:
for batch in train_loader:
    inputs, labels = batch
    break

In [19]:
inputs

tensor([[[ 0.4485,  0.4405,  0.4325,  0.4325,  0.4485],
         [-0.6974, -0.5753, -0.5595, -0.6068, -0.5910],
         [-0.2545, -0.2398, -0.2691, -0.1555, -0.1555],
         [ 0.6105,  0.3508,  0.2989,  0.1950,  0.1431],
         [-0.7842, -0.9158, -0.9158, -0.9378, -0.9597],
         [-0.7454, -0.9278, -0.9278, -0.9278, -0.9278],
         [ 1.1831,  0.9624,  0.9073,  0.8797,  0.8521],
         [-1.3012, -1.4474, -1.4474, -1.4474, -1.4474]]])

In [30]:
cnn_weight = model.cnn[0].weight[0].detach().cpu()

In [32]:
output = model.cnn[0](inputs.to(device)).detach().cpu()

In [38]:
cnn_weight, inputs, output[0, 0]

(tensor([[-19.6986, -21.7953, -27.0025],
         [ 44.4133,  44.1890,  44.3681],
         [-11.4387,  -3.3286,   3.5227],
         [-41.0221, -40.3241, -40.9374],
         [ 32.3187,  32.3442,  31.1909],
         [-41.2163, -41.4004, -41.5270],
         [ -0.6060,  -0.5368,  -0.0994],
         [ 25.2135,  24.7958,  24.0633]]),
 tensor([[[ 0.4485,  0.4405,  0.4325,  0.4325,  0.4485],
          [-0.6974, -0.5753, -0.5595, -0.6068, -0.5910],
          [-0.2545, -0.2398, -0.2691, -0.1555, -0.1555],
          [ 0.6105,  0.3508,  0.2989,  0.1950,  0.1431],
          [-0.7842, -0.9158, -0.9158, -0.9378, -0.9597],
          [-0.7454, -0.9278, -0.9278, -0.9278, -0.9278],
          [ 1.1831,  0.9624,  0.9073,  0.8797,  0.8521],
          [-1.3012, -1.4474, -1.4474, -1.4474, -1.4474]]]),
 tensor([-231.8735, -211.1289, -205.0359]))

In [98]:
cnn_weight.detach().cpu().numpy().reshape(-1)

array([-19.698553  , -21.795269  , -27.002476  ,  44.41327   ,
        44.189     ,  44.368053  , -11.438744  ,  -3.3285847 ,
         3.5227225 , -41.022118  , -40.324123  , -40.937428  ,
        32.318653  ,  32.344234  ,  31.190908  , -41.216343  ,
       -41.400406  , -41.52702   ,  -0.606003  ,  -0.5368076 ,
        -0.09939746,  25.213467  ,  24.795797  ,  24.063305  ],
      dtype=float32)

In [95]:
torch.sum(torch.mul(inputs[0, :, :3], cnn_weight)) + model.cnn[0].bias[0]

tensor(-231.8735, device='cuda:0', grad_fn=<AddBackward0>)

In [137]:
def tensor_to_hex_list(tensor: torch.Tensor, float_bits: int, bits: int):
    arr = tensor.detach().cpu().numpy()
    arr = arr * 2 ** float_bits
    arr = np.array(arr.reshape(-1), dtype=np.int64)
    hex_list =  [f"{bits}'h" + hex(abs(i))[2:].rjust(int(bits/4), "0") if i > 0 else f"-{bits}'h" + hex(abs(i))[2:].rjust(int(bits/4), "0") for i in arr]
    
    return hex_list

In [116]:
cnn_weight_hex = tensor_to_hex_list(cnn_weight, 8, 16)
inputs_hex = tensor_to_hex_list(inputs, 8, 16)

In [117]:
for i in cnn_weight_hex:
    print(f"{i},")

-16'h13b2,
-16'h15cb,
-16'h1b00,
16'h2c69,
16'h2c30,
16'h2c5e,
-16'h0b70,
-16'h0354,
16'h0385,
-16'h2905,
-16'h2852,
-16'h28ef,
16'h2051,
16'h2058,
16'h1f30,
-16'h2937,
-16'h2966,
-16'h2986,
-16'h009b,
-16'h0089,
-16'h0019,
16'h1936,
16'h18cb,
16'h1810,


In [118]:
for i in inputs_hex:
    print(f"{i},")

16'h0072,
16'h0070,
16'h006e,
16'h006e,
16'h0072,
-16'h00b2,
-16'h0093,
-16'h008f,
-16'h009b,
-16'h0097,
-16'h0041,
-16'h003d,
-16'h0044,
-16'h0027,
-16'h0027,
16'h009c,
16'h0059,
16'h004c,
16'h0031,
16'h0024,
-16'h00c8,
-16'h00ea,
-16'h00ea,
-16'h00f0,
-16'h00f5,
-16'h00be,
-16'h00ed,
-16'h00ed,
-16'h00ed,
-16'h00ed,
16'h012e,
16'h00f6,
16'h00e8,
16'h00e1,
16'h00da,
-16'h014d,
-16'h0172,
-16'h0172,
-16'h0172,
-16'h0172,


In [82]:
inputs

tensor([[[ 0.4485,  0.4405,  0.4325,  0.4325,  0.4485],
         [-0.6974, -0.5753, -0.5595, -0.6068, -0.5910],
         [-0.2545, -0.2398, -0.2691, -0.1555, -0.1555],
         [ 0.6105,  0.3508,  0.2989,  0.1950,  0.1431],
         [-0.7842, -0.9158, -0.9158, -0.9378, -0.9597],
         [-0.7454, -0.9278, -0.9278, -0.9278, -0.9278],
         [ 1.1831,  0.9624,  0.9073,  0.8797,  0.8521],
         [-1.3012, -1.4474, -1.4474, -1.4474, -1.4474]]])

In [85]:
tensor_to_hex_list(model.cnn[0].bias, 8)

["16'h08d8",
 "16'h0b7e",
 "-16'h0b01",
 "16'h1462",
 "16'h23ea",
 "-16'h1e79",
 "16'h27a7",
 "-16'h03b3",
 "-16'h28a4",
 "16'h2752"]

In [86]:
model.cnn[0].bias

Parameter containing:
tensor([  8.8471,  11.4942, -11.0057,  20.3848,  35.9173, -30.4756,  39.6558,
         -3.6994, -40.6424,  39.3213], device='cuda:0', requires_grad=True)

In [104]:
model.fc[0].bias

Parameter containing:
tensor([-42.5195,   5.7998,  27.6298, -23.9818,  24.4148,  29.1349,  -8.1405,
         10.6412,   6.3522, -41.3713, -31.6411,  27.3438, -43.1333, -39.3329,
         45.0211, -18.4086, -38.9969, -11.5696,  -7.3386,  18.2612,  16.9376,
         21.9714, -22.5964, -36.8528, -23.7138,  34.5924, -44.0687],
       device='cuda:0', requires_grad=True)

In [105]:
output

tensor([[[-231.8735, -211.1289, -205.0359],
         [ 213.3433,  217.3872,  214.7934],
         [  90.6032,   81.7818,   77.9445],
         [ -32.1318,  -50.9191,  -53.1582],
         [ 503.1721,  494.3076,  486.4103],
         [ 143.4365,  121.3317,  112.4147],
         [ 205.8760,  194.6766,  190.9556],
         [-231.0771, -240.9203, -247.9731],
         [-362.3427, -356.7926, -352.1190],
         [-192.4772, -179.5437, -172.2068]]])

In [112]:
model.fc[0](model.cnn[2](output).to(device))

tensor([[-38186.6094,  18719.0820, -41925.6133,   9504.1885, -16845.1621,
          15184.8652, -13563.2441,  37538.7188,  -7083.8740, -23894.3828,
           -115.7192,  15192.9238, -51408.7031,  34503.4609, -37975.0273,
          -5718.3486,  88901.7812,  53196.6719, -35372.0273, -14858.5732,
          35911.5508,  38343.9844, -18380.5645, -10531.6348,  -3204.3152,
           9574.7754, -18521.1895]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [110]:
labels

tensor([16])

In [136]:
output, model.fc[0].weight[0]

(tensor([[[-231.8735, -211.1289, -205.0359],
          [ 213.3433,  217.3872,  214.7934],
          [  90.6032,   81.7818,   77.9445],
          [ -32.1318,  -50.9191,  -53.1582],
          [ 503.1721,  494.3076,  486.4103],
          [ 143.4365,  121.3317,  112.4147],
          [ 205.8760,  194.6766,  190.9556],
          [-231.0771, -240.9203, -247.9731],
          [-362.3427, -356.7926, -352.1190],
          [-192.4772, -179.5437, -172.2068]]]),
 tensor([  9.9009,   6.7030,  12.8561, -17.1859, -15.4959, -14.5686,  10.5488,
           9.6615,  11.1175,  13.5918,  12.2274,  12.0412, -13.4970, -10.8626,
          -7.1948,   8.3417,   8.2107,  12.0162,  -9.4247,  -9.1299,  -9.3204,
          -7.0088,  -6.6695,  -4.7617,  20.4395,  20.8245,  19.5263, -21.5311,
         -22.3276, -24.5059], device='cuda:0', grad_fn=<SelectBackward0>))

In [138]:
fc_input_hex = tensor_to_hex_list(output, 8, 24)
fc_weight_hex = tensor_to_hex_list(model.fc[0].weight[0], 8, 16)

In [139]:
for i in fc_input_hex:
    print(f"{i},")

-24'h00e7df,
-24'h00d320,
-24'h00cd09,
24'h00d557,
24'h00d963,
24'h00d6cb,
24'h005a9a,
24'h0051c8,
24'h004df1,
-24'h002021,
-24'h0032eb,
-24'h003528,
24'h01f72c,
24'h01ee4e,
24'h01e669,
24'h008f6f,
24'h007954,
24'h00706a,
24'h00cde0,
24'h00c2ad,
24'h00bef4,
-24'h00e713,
-24'h00f0eb,
-24'h00f7f9,
-24'h016a57,
-24'h0164ca,
-24'h01601e,
-24'h00c07a,
-24'h00b38b,
-24'h00ac34,


In [132]:
for i in fc_weight_hex:
    print(f"{i},")

16'h09e6,
16'h06b3,
16'h0cdb,
-16'h112f,
-16'h0f7e,
-16'h0e91,
16'h0a8c,
16'h09a9,
16'h0b1e,
16'h0d97,
16'h0c3a,
16'h0c0a,
-16'h0d7f,
-16'h0adc,
-16'h0731,
16'h0857,
16'h0835,
16'h0c04,
-16'h096c,
-16'h0921,
-16'h0952,
-16'h0702,
-16'h06ab,
-16'h04c2,
16'h1470,
16'h14d3,
16'h1386,
-16'h1587,
-16'h1653,
-16'h1881,


In [124]:
fc_bias_hex = tensor_to_hex_list(model.fc[0].bias, 8, 16)

In [133]:
model.fc[0].bias

Parameter containing:
tensor([-42.5195,   5.7998,  27.6298, -23.9818,  24.4148,  29.1349,  -8.1405,
         10.6412,   6.3522, -41.3713, -31.6411,  27.3438, -43.1333, -39.3329,
         45.0211, -18.4086, -38.9969, -11.5696,  -7.3386,  18.2612,  16.9376,
         21.9714, -22.5964, -36.8528, -23.7138,  34.5924, -44.0687],
       device='cuda:0', requires_grad=True)

In [127]:
fc_bias_hex

["-16'h2a85",
 "16'h05cc",
 "16'h1ba1",
 "-16'h17fb",
 "16'h186a",
 "16'h1d22",
 "-16'h0823",
 "16'h0aa4",
 "16'h065a",
 "-16'h295f",
 "-16'h1fa4",
 "16'h1b58",
 "-16'h2b22",
 "-16'h2755",
 "16'h2d05",
 "-16'h1268",
 "-16'h26ff",
 "-16'h0b91",
 "-16'h0756",
 "16'h1242",
 "16'h10f0",
 "16'h15f8",
 "-16'h1698",
 "-16'h24da",
 "-16'h17b6",
 "16'h2297",
 "-16'h2c11"]

In [18]:
with open("../dataset/train/1/A.txt", "r") as f:
    lines = f.readlines()[:5]

In [19]:
data = []

for line in lines:
    line = line.strip().split()
    for n in line:
        data.append(int(n))

In [20]:
for i in data:
    print(f"{i},")

1012,
19,
-3,
303,
398,
403,
392,
347,
1015,
23,
-3,
301,
397,
402,
395,
354,
1012,
23,
-11,
300,
394,
399,
391,
350,
1012,
15,
-7,
301,
396,
401,
393,
351,
1019,
31,
3,
301,
396,
401,
393,
350,
